In [ ]:
import numpy as np
from scipy.signal import convolve

In [ ]:
def sfb(y, g0, g1):
    """
    Synthesis Filter Bank (SFB)
    
    Parameters:
    y : output from 'afb' function
    g0, g1 : synthesis filters
    
    Returns:
    x : reconstructed input
    """
    # Upsample the signals
    x0 = np.zeros(2 * len(y[0]), dtype=y.dtype)
    x1 = np.zeros(2 * len(y[0]), dtype=y.dtype)

    x0[::2] = y[0]
    x1[::2] = y[1]

    # Convolve with the synthesis filters
    temp = convolve(x0, g0, mode='full')
    temp[:len(temp) - len(x0)] += temp[len(x0):]
    take = temp[:len(x0)]
    shift = len(g0)
    take0 = take[np.mod(np.arange(len(x0)) + shift - 1, len(x0))]

    temp = convolve(x1, g1, mode='full')
    temp[:len(temp) - len(x1)] += temp[len(x1):]
    take = temp[:len(x1)]
    shift = len(g1)
    take1 = take[np.mod(np.arange(len(x1)) + shift - 1, len(x1))]

    return take0 + take1


In [ ]:
def IDTWPT(y, h_first, h, f):
    """
    Inverse Dual-Tree Wavelet Packet Transform (IDTWPT)
    
    Parameters:
    y : the cell array arranged as in DTWPT
    h_first : first stage synthesis filters ([h0_first, h1_first])
    h : dual-tree synthesis filters ([h0, h1])
    f : the 'same' synthesis filters ([f0, f1])
    
    Returns:
    x : reconstructed input signal
    """
    max_level = int(np.log2(len(y)))

    xx = y[0, :].tolist()

    for n in range(max_level, 2, -1):
        x2 = [None] * (2**(n - 1))
        for k in range(1, 2**(n - 1) + 1):
            if k % 2**(n - 2) == 1:
                fil0 = h[0, :]
                fil1 = h[1, :]
            else:
                fil0 = f[0, :]
                fil1 = f[1, :]
            x2[k - 1] = sfb([xx[2 * k - 2], xx[2 * k - 1]], fil0, fil1)
        xx = x2

    # Second stage
    fil0 = h[0, :]
    fil1 = h[1, :]
    x2 = [None] * 2
    x2[0] = sfb([xx[0], xx[1]], fil0, fil1)
    x2[1] = sfb([xx[2], xx[3]], fil0, fil1)

    # First stage
    fil0 = h_first[0, :]
    fil1 = h_first[1, :]
    x = sfb([x2[0], x2[1]], fil0, fil1)

    return x


In [ ]:
# Example input data and filter coefficients
y = np.random.rand(1, 8)  # Replace with actual coefficients
h_first = np.array([[0.5, 0.5], [-0.5, 0.5]])
h = np.array([[0.5, 0.5], [-0.5, 0.5]])
f = np.array([[0.5, 0.5], [-0.5, 0.5]])

# Perform IDTWPT
x_reconstructed = IDTWPT(y, h_first, h, f)
print(x_reconstructed)
